# Final processing script, to see the results of the classification analyses

This script is quite straightforward. It loads the relevant classificaiton scores for each condition and concatenates them. It also identifies clusters that are statistically significant (after permutation testing). The results are all plotted below. 

Critically, the data is loaded through the function "get_accuracy_by_time_guggenmos_v2_reref" which is located in EEG_auxiliary_module_sptm_wICA.ipynb. You will need to adjust this function to load the output from "TemporallySpecificAnalyses_SVC_mainFunctions", depending on how you arrange your folder system.

#### Import relevant modules

In [1]:
%reload_ext autoreload
%autoreload 2
%run EEG_auxiliary_module_sptm_wICA.ipynb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
import os
import mne
from IPython.core.debugger import set_trace
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    os.environ['PYTHONWARNINGS']='ignore'
from scipy import stats as stats


C:\Users\bachmanm\AppData\Local\Temp\ipykernel_15352\1428617302.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### get accuracy for all subjects

In [20]:
## creates a ribbonplot for each classification allowing to compare the timing 
## of accuracies across subjects"""

## Adjust these variables for your conditions of interest.
conds    = ['Faces','FaceValues','Colors','ColorValues','Value-2','Response-0']#,

## Nothing below this line needs to be changed.
subjects = [str(s) for s in [150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,173,174,176,177,178,180,181,182,183,185,186,188,189,190,191,192,193,194,195]]
events   = ['Stimulus_long']#

tres = .05
%matplotlib qt

# for multiclass in multiclassValues:
multiclass = 'binary'    
if multiclass == 'multi':
    chance = 20
elif multiclass == 'binary':
    chance = 50

    ################# get accuracy by time #####################  
fig, ax = plt.subplots(figsize=(10,10))
n_cols  = len(events)
n_rows  = len(conds)

n_panel = 0

for condition in conds:

    for event in events:
        if event.lower() == 'stimulus':
            time = np.arange(-200,1000,10)
        elif event.lower() == 'dot':
            time = np.arange(800,2000,10)
        elif event.lower() == 'stimulus_long':
            time = np.arange(-200,2000,10)

        if condition == 'Response':
            chance = 50


        allAccuracy = np.full((len(subjects),time.shape[0]),np.nan)

        for s,subj in enumerate(subjects):


            accuracy = get_accuracy_by_time_guggenmos_v2_reref(subj,condition, event, 10, multiclass= multiclass,jitFix = None)
            accuracy = np.nanmean(accuracy, axis=(0,1))

            allAccuracy[s,:] = accuracy

        t,pval2 = stats.ttest_1samp(allAccuracy, chance)
        t_obs, clusters, cluster_p_values, hzero = mne.stats.spatio_temporal_cluster_1samp_test(allAccuracy-chance)
        print(cluster_p_values)
        good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
        good_clusters = [clusters[idx] for idx in good_clusters_idx]
        
        sigLine2    = np.empty(time.shape)
        sigLine2[:] = np.nan
        for x in good_clusters:
            sigLine2[x] = 60
        n_panel +=1
        xticks = np.around(np.linspace(time[0],time[-1],10),decimals = 2)
        timelabels = []
        [timelabels.append(str(t)) for t in xticks]

        ax = plt.subplot(n_rows,n_cols,n_panel)        
        sigLine    = np.empty(time.shape)
        sigLine[:] = np.nan
        sigLine[(pval2<=tres) & (t>0)] = 62
        # calculate average
        accAvg = list(np.mean(allAccuracy,0))
        plt.axhline(y=chance,color='k',linestyle='-')
        if event.lower() == 'stimulus':
            plt.xlim(-200,1000)
            plt.axvline(x=0,color='k',linestyle='--')
            plt.show
        elif event.lower() == 'dot':
            plt.xlim(800,2000)
            plt.axvline(x=1000,color='k',linestyle='--')
            plt.show
        elif event.lower() == 'stimulus_long':
            plt.xlim(-200,2000)
            tick_labels = ['-200','0','200','400','600','800','1000','1200','1400','1600','1800','2000']
            plt.axvline(x=1000,color='k',linestyle='--')
            plt.axvline(x=0,color='k',linestyle='--')
            plt.xticks(np.arange(time[0],time[-1],step=200))
            plt.show
        plt.plot(time,accAvg)
        plt.plot(time, sigLine2,marker='s',  markersize=3,c='red')
        plt.xlabel('time (ms)')
        plt.ylabel('accuracy')
        plt.ylim([chance - 5, chance + 15])
        plt.yticks(np.arange(chance-5,chance+15,step=5))
        plt.title(condition+event)

        

    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.3, hspace=.8)
    fig.suptitle(multiclass)
    print(condition)
    plt.savefig('full_set_n40_permutations_reref.eps', format='eps')


Using a threshold of 2.022691
stat_fun(H1): min=-1.793382 max=6.364985
Running initial clustering …
Found 9 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.95800781 0.00097656 0.0703125  0.8046875  0.95117188 0.7734375
 0.72753906 0.40917969 0.4375    ]
Faces
Using a threshold of 2.022691
stat_fun(H1): min=-2.336137 max=4.798401
Running initial clustering …
Found 9 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.6796875  0.57226562 0.95996094 0.00195312 0.95898438 0.578125
 0.95996094 0.03417969 0.85351562]
FaceValues
Using a threshold of 2.022691
stat_fun(H1): min=-2.095269 max=9.655751
Running initial clustering …
Found 9 clusters


...

[0.00097656 0.11621094 0.39746094 0.93945312 0.63378906 0.66796875
 0.37988281 0.78027344 0.94042969]
Colors
Using a threshold of 2.022691
stat_fun(H1): min=-1.503737 max=5.514955
Running initial clustering …
Found 21 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.01953125 0.00878906 0.72265625 0.00585938 0.05761719 0.06347656
 0.64257812 0.609375   0.93164062 0.1796875  0.67382812 0.04492188
 0.68457031 0.41992188 0.89257812 0.16894531 0.95996094 0.95703125
 0.86132812 0.95605469 0.40527344]
ColorValues
Using a threshold of 2.022691
stat_fun(H1): min=-2.160307 max=4.606823
Running initial clustering …
Found 25 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.85449219 0.32226562 0.39453125 0.97167969 0.59863281 0.50976562
 0.21289062 0.76855469 0.01367188 0.35449219 0.76464844 0.60644531
 0.39648438 0.03320312 0.53515625 0.11816406 0.1953125  0.01660156
 0.04199219 0.02636719 0.96386719 0.7890625  0.60644531 0.94335938
 0.92773438]
Value-2


In [1]:
# This shows the significant time clusters for the last condition run.
time[~np.isnan(sigLine2)]

NameError: name 'time' is not defined